In [1]:
import pandas as pd
import requests
import json
import numpy as np
import sys
import pickle
import multiprocessing as mp
from datetime import datetime, timezone, timedelta
from pathlib import Path
from src.stats_scraper import StatsScraper

In [2]:
ss = StatsScraper("../data/")

In [1]:
import hockey_scraper

In [2]:
hockey_scraper.scrape_seasons([2007], n_jobs=10)

aping Game  2007020970 2008-02-28
Scraping Game  2007020838 2008-02-09
Scraping Game  2007020773 2008-01-31
Scraping Game  2007020872 2008-02-14
Scraping Game  2007020905 2008-02-19
Scraping Game  2007020676 2008-01-15
Scraping Game  2007020741 2008-01-24
Scraping Game  2007020938 2008-02-23
Scraping Game  2007020708 2008-01-19
Scraping Game  2007020806 2008-02-05
Scraping Game  2007020971 2008-02-29
Scraping Game  2007020840 2008-02-09
Scraping Game  2007020873 2008-02-14
Scraping Game  2007020774 2008-01-31
Scraping Game  2007020906 2008-02-19
Scraping Game  2007020677 2008-01-15
Scraping Game  2007020742 2008-01-24
Scraping Game  2007020939 2008-02-23
Scraping Game  2007020807 2008-02-05
Scraping Game  2007020709 2008-01-19
Scraping Game  2007020972 2008-02-29
Scraping Game  2007020874 2008-02-14
Scraping Game  2007020841 2008-02-09
Scraping Game  2007020775 2008-01-31
Scraping Game  2007020678 2008-01-15
Scraping Game  2007020907 2008-02-19
Scraping Game  2007020744 2008-01-24
Scra

In [3]:
for i_season in range(2010, 2019):
    ss.download_season(i_season, n_jobs=10)

## StatsExtractor: download SEASON 2010/2011 from NHL API
... saved [64.31 s] to '../data/games_raw/2010-2011.pkl'
## StatsExtractor: download SEASON 2011/2012 from NHL API
... saved [55.26 s] to '../data/games_raw/2011-2012.pkl'
## StatsExtractor: download SEASON 2012/2013 from NHL API
... saved [23.25 s] to '../data/games_raw/2012-2013.pkl'
## StatsExtractor: download SEASON 2013/2014 from NHL API
... saved [53.90 s] to '../data/games_raw/2013-2014.pkl'
## StatsExtractor: download SEASON 2014/2015 from NHL API
... saved [50.35 s] to '../data/games_raw/2014-2015.pkl'
## StatsExtractor: download SEASON 2015/2016 from NHL API
... saved [51.85 s] to '../data/games_raw/2015-2016.pkl'
## StatsExtractor: download SEASON 2016/2017 from NHL API
... saved [45.80 s] to '../data/games_raw/2016-2017.pkl'
## StatsExtractor: download SEASON 2017/2018 from NHL API
... saved [40.81 s] to '../data/games_raw/2017-2018.pkl'
## StatsExtractor: download SEASON 2018/2019 from NHL API
... saved [42.76 s] to

In [4]:
with open("../data/games_raw/2010-2011.pkl", "rb") as f:
    games = pickle.load(f)
games[0]

-56.0, 'y': -33.0},
   'period': 2,
   'time': '17:05',
   'shotType': 'Wrist Shot'},
  {'type': 'SHOT',
   'team': {'id': 8,
    'name': 'Montréal Canadiens',
    'link': '/api/v1/teams/8',
    'triCode': 'MTL'},
   'coordinates': {'x': -83.0, 'y': -17.0},
   'period': 2,
   'time': '17:10',
   'shotType': 'Snap Shot'},
  {'type': 'SHOT',
   'team': {'id': 10,
    'name': 'Toronto Maple Leafs',
    'link': '/api/v1/teams/10',
    'triCode': 'TOR'},
   'coordinates': {'x': 57.0, 'y': -37.0},
   'period': 2,
   'time': '17:35',
   'shotType': 'Wrist Shot'},
  {'type': 'HIT',
   'team': {'id': 8,
    'name': 'Montréal Canadiens',
    'link': '/api/v1/teams/8',
    'triCode': 'MTL'},
   'coordinates': {'x': 93.0, 'y': -31.0},
   'period': 2,
   'time': '17:40'},
  {'type': 'BLOCKED_SHOT',
   'team': {'id': 10,
    'name': 'Toronto Maple Leafs',
    'link': '/api/v1/teams/10',
    'triCode': 'TOR'},
   'coordinates': {'x': -59.0, 'y': 2.0},
   'period': 2,
   'time': '17:49'},
  {'type': '

In [153]:
%%time
games = []
for game_num in range(1, max_game_num):
    game_url = base_url + f"game/{season}{game_type:02d}{game_num:04d}/feed/live"
    response = requests.get(game_url)
    if response.status_code != 200:
        print()
        print(f"!! Game Nr. {game_num}: {response}")
        break  
    print(f"\rGame Nr. {game_num}: {response}", end="")
    games.append(response.json())

Game Nr. 11: <Response [200]>

KeyboardInterrupt: 

In [ ]:
len(games[0])

In [59]:
game_num = 1
season = 2013
base_url = "https://statsapi.web.nhl.com/api/v1/"
game_url = base_url + f"game/{season}02{game_num:04d}/feed/live"
response = requests.get(game_url)
game_json = response.json()
game_json.keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

In [60]:
print(game_json['gameData']['teams']['home']['name'], "-", game_json['gameData']['teams']['away']['name'])

Montréal Canadiens - Toronto Maple Leafs


In [61]:
dt = datetime.strptime(game_json['gameData']['datetime']['dateTime'], "%Y-%m-%dT%H:%M:%SZ")
dt.astimezone(timezone(timedelta(hours=-4)))

datetime.datetime(2013, 10, 1, 17, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000)))

In [106]:
game_json['liveData']['plays']['allPlays'][3].keys()

dict_keys(['players', 'result', 'about', 'coordinates', 'team'])

In [99]:
game_json["liveData"]['plays']['allPlays'][56]['result']['eventTypeId']

'PENALTY'

In [112]:
result = {}
result['id'] = game_json['gamePk']
result['datetime'] = game_json['gameData']['datetime']['dateTime']
result['teams'] = {}
for team in ['home', 'away']:
    result['teams'][team] = {}
    for key in ['id', 'name', 'triCode']:
        result['teams'][team][key] = game_json['gameData']['teams'][team][key]
    result['teams'][team]['teamStats'] = game_json['liveData']['boxscore']['teams'][team]['teamStats']['teamSkaterStats']
result['plays'] = []
for play in game_json['liveData']['plays']['allPlays']:
    if bool(play['coordinates']):
        result['plays'].append({
            'type': play['result']['eventTypeId'],
            'team': play['team'],
            'coordinates': play['coordinates']
        })
result['teams']

{'home': {'id': 8,
  'name': 'Montréal Canadiens',
  'triCode': 'MTL',
  'teamStats': {'goals': 3,
   'pim': 53,
   'shots': 37,
   'powerPlayPercentage': '0.0',
   'powerPlayGoals': 0.0,
   'powerPlayOpportunities': 4.0,
   'faceOffWinPercentage': '47.3',
   'blocked': 15,
   'takeaways': 7,
   'giveaways': 10,
   'hits': 21}},
 'away': {'id': 10,
  'name': 'Toronto Maple Leafs',
  'triCode': 'TOR',
  'teamStats': {'goals': 4,
   'pim': 47,
   'shots': 38,
   'powerPlayPercentage': '14.3',
   'powerPlayGoals': 1.0,
   'powerPlayOpportunities': 7.0,
   'faceOffWinPercentage': '52.7',
   'blocked': 17,
   'takeaways': 3,
   'giveaways': 5,
   'hits': 25}}}

In [22]:
df = pd.read_pickle(f"../data/odds/{season}-{season+1}.pkl")
df

,date,home,away,result,1,X,2
0,2013-10-01 19:00:00,Montreal Canadiens,Toronto Maple Leafs,2,2.26,3.89,2.70
1,2013-10-01 20:00:00,Chicago Blackhawks,Washington Capitals,1,1.93,4.09,3.30
2,2013-10-01 22:00:00,Edmonton Oilers,Winnipeg Jets,2,2.11,3.88,2.96
3,2013-10-02 19:30:00,Philadelphia Flyers,Toronto Maple Leafs,2,2.24,4.01,2.71
4,2013-10-02 20:00:00,Detroit Red Wings,Buffalo Sabres,1,1.78,4.17,3.77
...,...,...,...,...,...,...,...
1225,2014-04-13 19:30:00,Pittsburgh Penguins,Ottawa Senators,X,1.92,4.05,3.29
1226,2014-04-13 20:00:00,Minnesota Wild,Nashville Predators,2,1.95,3.89,3.36
1227,2014-04-13 20:00:00,Anaheim Ducks,Colorado Avalanche,X,2.13,3.85,2.94
1228,2014-04-13 21:00:00,Vancouver Canucks,Calgary Flames,1,2.05,3.76,3.16


In [25]:
df.iloc[0, 0].to_pydatetime().astimezone(timezone(timedelta(hours=+2)))

datetime.datetime(2013, 10, 1, 19, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))

In [32]:
df.iloc[0, 0].to_pydatetime().date() == dt.date()

True

In [31]:
dt.date()

datetime.date(2013, 10, 1)

In [39]:
plays_len = len(game_json['liveData']['plays']['allPlays'])
for i in range(plays_len):
    event_type = game_json['liveData']['plays']['allPlays'][i]['result']['eventTypeId']
    coords = game_json['liveData']['plays']['allPlays'][i]['coordinates']
    if "HIT" in event_type:
        print(f"{event_type}: {coords}")
        print(game_json['liveData']['plays']['allPlays'][i]['result']['description'])
        print(game_json['liveData']['plays']['allPlays'][i]['team'])


HIT: {'x': -97.0, 'y': 20.0}
Alex Galchenyuk hit Jake Gardiner
{'id': 8, 'name': 'Montréal Canadiens', 'link': '/api/v1/teams/8', 'triCode': 'MTL'}
HIT: {'x': 91.0, 'y': 39.0}
Jarred Tinordi hit Colton Orr
{'id': 8, 'name': 'Montréal Canadiens', 'link': '/api/v1/teams/8', 'triCode': 'MTL'}
HIT: {'x': 97.0, 'y': 14.0}
Jay McClement hit Francis Bouillon
{'id': 10, 'name': 'Toronto Maple Leafs', 'link': '/api/v1/teams/10', 'triCode': 'TOR'}
HIT: {'x': 95.0, 'y': -36.0}
Brandon Prust hit Carter Ashton
{'id': 8, 'name': 'Montréal Canadiens', 'link': '/api/v1/teams/8', 'triCode': 'MTL'}
HIT: {'x': 51.0, 'y': 39.0}
P.K. Subban hit Tyler Bozak
{'id': 8, 'name': 'Montréal Canadiens', 'link': '/api/v1/teams/8', 'triCode': 'MTL'}
HIT: {'x': -96.0, 'y': -15.0}
Alex Galchenyuk hit Paul Ranger
{'id': 8, 'name': 'Montréal Canadiens', 'link': '/api/v1/teams/8', 'triCode': 'MTL'}
HIT: {'x': -92.0, 'y': -33.0}
George Parros hit Mark Fraser
{'id': 8, 'name': 'Montréal Canadiens', 'link': '/api/v1/teams/8